# Table of contents
* [1. Abstract](#0)
* [2. Setup](#1)
  * [2.1 Start Hadoop](#1.1)  
  * [2.2 Create SparkSession](#1.2)
* [3. Creation of the dataframe & Data Cleaning](#2)
* [4. Insights](#3)
  * [4.1 Insight 1: Average rating of apps on Google Play Store](#3.1)
  * [4.2 Insight 2: Average price of paid apps on Google Play Store per category](#3.2)
  * [4.3 Insight 3: The strange case of finance apps prices](#3.3)
  * [4.4 Insight 4: Average size and price of top 1% rated apps](#3.4)
  * [4.4 Insight 4: Common words in positive sentiment reviews](#3.5)
* [5. Conclusions](#4)


<a id='0'></a>
# 1. Abstract & Tools

The objective of this project is to get deeper into a real data problem from the technical point of view in order to learn and understand a potential solution, using Big Data, Data Engineering and Analytics tools. This was done using NiFi for data ingestion, HDFS for storage, and Spark for processing. The first step of the project was to use NiFi to ingest the data from various sources such as CSV files, (web scraped data of 10k Play Store apps for analysing the Android market) into HDFS. This ensured that the data was properly formatted before being stored in HDFS for further processing. Once the data was stored in HDFS, I used Spark to process and analyze the data. I used Spark SQL to perform SQL-like operations on the data and Spark DataFrames to perform various data transformations and aggregations. Finally, I used Spark to generate various insights and visualizations from the processed data. I used Spark SQL to generate various statistics and Spark DataFrames to generate various insights.Overall, this project allowed me to gain hands-on experience in using NiFi, HDFS, and Spark to process and analyze large datasets. Additionally, I was able to gain valuable insights from the data, which could be used to inform decision-making and generate business value. The Play Store apps data has enormous potential to drive app-making businesses to success. Actionable insights can be drawn for developers or app owners to work on and increase the performance of their current and futures apps. The following analysis will focus on describing the current status, trends and statistics in the market.

<center><img src="Visualresource.png"></center>

<a id='1'></a>
# 2. Setup

<a id='1.1'></a>
### 2.1 Start Hadoop

Start Hadoop

Open a terminal and execute
```sh
hadoop-start.sh
```

<a id='1.2'></a>
### 2.2 Create SparkSession

In [1]:
# Import libraries
import findspark
import pandas as pd
import os

# Search Spark installation
findspark.init()
pd.set_option('display.max_colwidth', None)
os.environ['PYSPARK_SUBMIT_ARGS']="--packages org.apache.spark:spark-avro_2.12:3.2.1,io.delta:delta-core_2.12:1.2.1 pyspark-shell"

In [2]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder\
            .appName("Google Play Store Project")\
            .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-55d2fd92-a428-4e7b-8936-022cee01099b;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.2.1 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 692ms :: artifacts dl 31ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storage;1.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.apache.spark#spark-avro_2.12;3.2.1 from central in [

<a id='2'></a>
# 3. Creation of the Dataframes & Data Cleaning

In [3]:
from pyspark.sql.functions import col
# DataFrame creation
Playstore = (spark.read.option("inferSchema", "true")
                    .option("header", "true")
                    .csv("hdfs://localhost:9000/datalake/raw/Googleplaystore/Playstore/"))
User_Reviews = (spark.read.option("inferSchema", "true")
                    .option("header", "true")
                    .csv("hdfs://localhost:9000/datalake/raw/Googleplaystore/User_Reviews/"))


In [4]:
# The inferred schema can be visualized using the printSchema() method
User_Reviews.printSchema()

root
 |-- App: string (nullable = true)
 |-- Translated_Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- Sentiment_Polarity: string (nullable = true)
 |-- Sentiment_Subjectivity: string (nullable = true)



In [5]:
# The inferred schema can be visualized using the printSchema() method
Playstore.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [6]:
# Cleaning the data, Renaming the columns to make sure that all the columns are without any “.” and “_” instead of space on both files.
def rename(column):
    if column=="#":
        return "id"
    else:
        return column.replace('.','').replace(' ', '_').lower()
    

In [7]:
User_Reviews_df= User_Reviews.select([col("`" + c + "`").alias(rename(c)) for c in User_Reviews.columns])
User_Reviews_df.show(5,True)

+--------------------+--------------------+--------------------+------------------+----------------------+
|                 app|   translated_review|           sentiment|sentiment_polarity|sentiment_subjectivity|
+--------------------+--------------------+--------------------+------------------+----------------------+
|10 Best Foods for...|"I like eat delic...| also ""Best Befo...|          Positive|                   1.0|
|10 Best Foods for...|This help eating ...|            Positive|              0.25|   0.28846153846153844|
|10 Best Foods for...|                 nan|                 nan|               nan|                   nan|
|10 Best Foods for...|Works great espec...|            Positive|               0.4|                 0.875|
|10 Best Foods for...|        Best idea us|            Positive|               1.0|                   0.3|
+--------------------+--------------------+--------------------+------------------+----------------------+
only showing top 5 rows



In [8]:
Playstore_df= Playstore.select([col("`" + c + "`").alias(rename(c)) for c in Playstore.columns])
Playstore_df.show(5,True)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 app|      category|rating|reviews|size|   installs|type|price|content_rating|              genres|    last_updated|       current_ver| android_ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

In [9]:
# Removing unneeded signs and characters to standardize the data formats, for changing the type of column from string to float. Also removing incoherent rows that were causing trouble
from pyspark.sql.functions import expr,regexp_replace
Playstore_df = Playstore_df.withColumn("price", expr("replace(price, '$', '')"))\
                        .withColumn("rating", expr("replace(rating, 'NaN', '0')"))\
                        .withColumn("size", expr("replace(size, 'M', '000000')"))\
                        .withColumn("size", expr("replace(size, 'k', '000')"))\
                        .withColumn("size", expr("replace(size, '.', '')"))\
                        .withColumn("size", expr("replace(size, 'Varies with device', '39000000')"))\
                
Playstore_df = Playstore_df.filter((Playstore_df.app != 'Life Made WI-Fi Touchscreen Photo Frame'))\
                            .filter((Playstore_df.app != '"Women""s Health Tips(Breast'))
                                                   
                                  

In [10]:
User_Reviews_df = User_Reviews_df.filter(User_Reviews_df.sentiment!='nan')

In [11]:
# Creating 2 functions to save the cleaned data as a parquet file
def save(df,f):
    (df.write.mode("overwrite")
            .format(f)
            .save(f"hdfs://localhost:9000/datalake/raw/Googleplaystore/Playstore.{f}/"))

[save(Playstore_df,f) for f in ["parquet"]]


[None]

In [12]:

def save2(df,f):
    (df.write.mode("overwrite")
            .format(f)
            .save(f"hdfs://localhost:9000/datalake/raw/Googleplaystore/User_Reviews.{f}/"))

[save2(User_Reviews_df,f) for f in ["parquet"]]


[None]

In [13]:
User_Reviews_df = spark.read.parquet("hdfs://localhost:9000/datalake/raw/Googleplaystore/User_Reviews.parquet")
Playstore_df = spark.read.parquet("hdfs://localhost:9000/datalake/raw/Googleplaystore/Playstore.parquet")

In [14]:
Playstore_df_clean = Playstore_df.drop("current_ver","android_ver","last_updated")
Playstore_df_clean.printSchema()

root
 |-- app: string (nullable = true)
 |-- category: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- reviews: string (nullable = true)
 |-- size: string (nullable = true)
 |-- installs: string (nullable = true)
 |-- type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- content_rating: string (nullable = true)
 |-- genres: string (nullable = true)



In [15]:
User_Reviews_clean = User_Reviews_df.drop("sentiment_subjectivity")
User_Reviews_clean.printSchema()

root
 |-- app: string (nullable = true)
 |-- translated_review: string (nullable = true)
 |-- sentiment: string (nullable = true)
 |-- sentiment_polarity: string (nullable = true)



In [16]:
from pyspark.sql.types import *
Playstore_df_clean = Playstore_df_clean \
  .withColumn("rating" ,
              Playstore_df["rating"]
              .cast(FloatType()))   \
  .withColumn("price" ,
              Playstore_df["price"]
              .cast(FloatType()))   \
  .withColumn("reviews" ,
              Playstore_df["reviews"]
              .cast(IntegerType()))   \
  
Playstore_df_clean.printSchema()

root
 |-- app: string (nullable = true)
 |-- category: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- reviews: integer (nullable = true)
 |-- size: string (nullable = true)
 |-- installs: string (nullable = true)
 |-- type: string (nullable = true)
 |-- price: float (nullable = true)
 |-- content_rating: string (nullable = true)
 |-- genres: string (nullable = true)



In [18]:
User_Reviews_clean = User_Reviews_clean \
  .withColumn("Sentiment_Polarity" ,
              User_Reviews_df["Sentiment_Polarity"]
              .cast(FloatType()))   \

User_Reviews_clean.printSchema()

root
 |-- app: string (nullable = true)
 |-- translated_review: string (nullable = true)
 |-- sentiment: string (nullable = true)
 |-- Sentiment_Polarity: float (nullable = true)



In [19]:
User_Reviews_clean_summary = User_Reviews_clean.summary()
User_Reviews_clean_summary.toPandas()

,summary,app,translated_review,sentiment,Sentiment_Polarity
0,count,37432,37427,37432,36784
1,mean,None,81759.0,None,0.183257177792301
2,stddev,None,None,None,0.35310687212555236
3,min,10 Best Foods for You,"!!!Dont waste time! Failed Samsung flagship phone galaxy s8, Installed ,shows rotating circle internet download, keeps rotates forever proper progress indication; finally shows failed download. Stupid game developers. Go NFS working good.","""""Bell""""",-1.0
4,25%,None,81759.0,None,0.0
5,50%,None,81759.0,None,0.15
6,75%,None,81759.0,None,0.40833333
7,max,Housing-Real Estate & Property,"搵楼租楼 A lot of time, a lot of time management, easy to take care of",that unbelievable!! Then,1.0


In [78]:
User_Reviews_clean
Playstore_df_clean.count()

10839

<a id='3'></a>
# 4. Insights

<a id='3.1'></a>
### 4.1 Insight 1: Average rating of apps on Google Play Store

In [38]:
# Finding the average rating of apps on Google Play Store:
import pyspark.sql.functions as F

mean= Playstore_df_clean.select(F.mean('rating').alias('mean'))

rating_mean = mean.collect()[0][0]
print (rating_mean)

3.6216460568237445


<a id='3.2'></a>
### 4.2 Insight 2: Average price of paid apps on Google Play Store per category

In [61]:
# Cheapest and most expensive categories on average (not including the free apps): Most expensive category: “Finance”, Cheapest category: “Libraries and Demo”
price_category = (Playstore_df_clean.where("price > 0").groupBy("category").mean("price").sort(F.desc("Avg(price)")))
price_category.show(50)

+-------------------+------------------+
|           category|        avg(price)|
+-------------------+------------------+
|            FINANCE|170.63705473787644|
|          LIFESTYLE|124.25631326123288|
|             EVENTS|109.98999786376953|
|           BUSINESS|13.233571120670863|
|            MEDICAL|13.210642084069208|
|             FAMILY|12.747539031567998|
|       PRODUCTIVITY|  8.96178584439414|
|        PHOTOGRAPHY| 6.100454460490834|
|MAPS_AND_NAVIGATION| 5.389999914169311|
|             SOCIAL| 5.323333263397217|
|          PARENTING| 4.789999961853027|
|          EDUCATION| 4.489999949932098|
|             DATING| 4.489999941417149|
|  AUTO_AND_VEHICLES| 4.489999930063884|
|BOOKS_AND_REFERENCE| 4.277499871594565|
|     FOOD_AND_DRINK| 4.239999890327454|
| HEALTH_AND_FITNESS| 4.208749920129776|
|             SPORTS|4.1666666467984514|
|   TRAVEL_AND_LOCAL| 4.162499924500783|
|            WEATHER| 4.052499920129776|
|      ENTERTAINMENT|3.9899998903274536|
|               

In [73]:
Playstore_df_clean.where("price > 0").groupBy("category").agg(F.count("price").alias("count"),F.mean("price")
                                                              .alias("avgprice")).sort(F.desc("avgprice")).show(50)

+-------------------+-----+------------------+
|           category|count|          avgprice|
+-------------------+-----+------------------+
|            FINANCE|   17|170.63705473787644|
|          LIFESTYLE|   19|124.25631326123288|
|             EVENTS|    1|109.98999786376953|
|           BUSINESS|   14|13.233571120670863|
|            MEDICAL|  109|13.210642084069208|
|             FAMILY|  191|12.747539031567998|
|       PRODUCTIVITY|   28|  8.96178584439414|
|        PHOTOGRAPHY|   22| 6.100454460490834|
|MAPS_AND_NAVIGATION|    5| 5.389999914169311|
|             SOCIAL|    3| 5.323333263397217|
|          PARENTING|    2| 4.789999961853027|
|          EDUCATION|    4| 4.489999949932098|
|             DATING|    7| 4.489999941417149|
|  AUTO_AND_VEHICLES|    3| 4.489999930063884|
|BOOKS_AND_REFERENCE|   28| 4.277499871594565|
|     FOOD_AND_DRINK|    2| 4.239999890327454|
| HEALTH_AND_FITNESS|   16| 4.208749920129776|
|             SPORTS|   24|4.1666666467984514|
|   TRAVEL_AN

<a id='3.3'></a>
### 4.3 Insight 3: The strange case of finance apps prices

170$ on average ? Seems odd, let's check these apps.

In [77]:
Playstore_df_clean.where("category == 'FINANCE'").sort(F.desc("price")).show(5)

+--------------------+--------+------+-------+--------+--------+----+------+--------------+-------+
|                 app|category|rating|reviews|    size|installs|type| price|content_rating| genres|
+--------------------+--------+------+-------+--------+--------+----+------+--------------+-------+
|   I Am Rich Premium| FINANCE|   4.1|   1867|47000000| 50,000+|Paid|399.99|      Everyone|Finance|
|           I am Rich| FINANCE|   4.3|    180|38000000|  5,000+|Paid|399.99|      Everyone|Finance|
|          I am Rich!| FINANCE|   3.8|     93|22000000|  1,000+|Paid|399.99|      Everyone|Finance|
|  I am rich(premium)| FINANCE|   3.5|    472|  965000|  5,000+|Paid|399.99|      Everyone|Finance|
|I am rich (Most e...| FINANCE|   4.1|    129|27000000|  1,000+|Paid|399.99|          Teen|Finance|
+--------------------+--------+------+-------+--------+--------+----+------+--------------+-------+
only showing top 5 rows



They are trash/troll apps, but people still buy them and their price is over the top.

<a id='3.4'></a>
### 4.4 Insight 4: Average size and price of top 1% rated apps

In [93]:
row_num = int(Playstore_df_clean.count()*0.01)

top_rated = Playstore_df_clean.sort(F.desc("rating")).take(row_num)


In [100]:
top_rated_df = spark.createDataFrame(top_rated)

In [104]:
top_rated_df.select(F.mean("price").alias("price_mean")).show()

+------------------+
|        price_mean|
+------------------+
|0.5937036960213272|
+------------------+



Average price of top 1% rated apps is 0.59$

In [105]:
top_rated_df.select(F.mean("size").alias("size_mean")).show()

+-------------------+
|          size_mean|
+-------------------+
|3.667187037037037E7|
+-------------------+



Average size of top 1% rated apps is 3.66MB

<a id='3.5'></a>
### 4.5 Insight 5: Common words in positive sentiment reviews

In [108]:
User_Reviews_clean.show(1)

+--------------------+--------------------+--------------------+------------------+
|                 app|   translated_review|           sentiment|Sentiment_Polarity|
+--------------------+--------------------+--------------------+------------------+
|10 Best Foods for...|"I like eat delic...| also ""Best Befo...|              null|
+--------------------+--------------------+--------------------+------------------+
only showing top 1 row



In [111]:
words = User_Reviews_clean.withColumn('wordCount', F.size(F.split(F.col('translated_review'), ' ')))
words.show()

+--------------------+--------------------+--------------------+------------------+---------+
|                 app|   translated_review|           sentiment|Sentiment_Polarity|wordCount|
+--------------------+--------------------+--------------------+------------------+---------+
|10 Best Foods for...|"I like eat delic...| also ""Best Befo...|              null|       16|
|10 Best Foods for...|This help eating ...|            Positive|              0.25|        7|
|10 Best Foods for...|Works great espec...|            Positive|               0.4|        6|
|10 Best Foods for...|        Best idea us|            Positive|               1.0|        3|
|10 Best Foods for...|            Best way|            Positive|               1.0|        2|
|10 Best Foods for...|             Amazing|            Positive|               0.6|        1|
|10 Best Foods for...|Looking forward app,|             Neutral|               0.0|        3|
|10 Best Foods for...|It helpful site !...|             Neut

In [118]:
User_Reviews_clean.where("sentiment == 'Positive'").withColumn('word', F.explode(F.split(F.col('translated_review'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show(50)

+------+-----+
|  word|count|
+------+-----+
|     I|21476|
|  game| 3389|
|  like| 3380|
|  good| 3008|
|   The| 2965|
|  love| 2903|
|    It| 2773|
|   get| 2721|
|  This| 2518|
| great| 2248|
| would| 2171|
|  time| 1962|
|really| 1959|
|  app.| 1907|
|  It's| 1887|
|   I'm| 1564|
| Great| 1502|
|  much| 1502|
|  many| 1492|
|  easy| 1437|
|   it.| 1436|
|  even| 1365|
|  make| 1284|
|   new| 1264|
|  I've| 1231|
|  best| 1217|
|   way| 1179|
| can't| 1164|
|  want| 1139|
|  need| 1133|
|better| 1122|
|  Very| 1118|
|   But| 1069|
|  Good| 1066|
|  play| 1034|
|   see| 1027|
| using| 1023|
| could| 1022|
|  Love| 1021|
|  also| 1013|
|  free| 1011|
|  able|  995|
| phone|  988|
| still|  916|
|   ads|  913|
|  work|  909|
| every|  907|
|update|  905|
|  nice|  905|
|   fun|  902|
+------+-----+
only showing top 50 rows



Most repeated words include: good, love, better, like , great, easy, best, free, nice, fun

<a id='4'></a>
# 5 Conclusions

In conclusion, the data analysis project found that the average rating of active apps on the Google Play Store is 3.62 stars out of 5. The cheapest paid app category is "Library & Demo" and the most expensive paid app category is "Finance." The top rated apps are found to have an average size of 3.66MB and an average price of $0.59. Furthermore, words that were frequently repeated in positive reviews include "Easy," "Love," and "Free." These findings can provide valuable insights for app developers and marketers to improve the performance of their apps on the Google Play Store.

<center><img src="Google-Play-Logo.png" width="500" height="300"></center>